In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from yaml import safe_load
import uuid

2022-10-28 13:52:11.619672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-28 13:52:11.731855: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-28 13:52:12.175969: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/israel/miniconda3/envs/tf/lib/
2022-10-28 13:52:12.176025: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plug

In [2]:
from StringProcessing import preprocessInput, encode_pad_tag, get_target_groups_from_connected_components

In [3]:
from SiameseModel import ContrastiveLoss, CosineDistance
from ModelEvaluation import find_Model_ranks, find_Gesalt_ranks

tf.version:  2.10.0
tf.keras.version:  2.10.0
tf.config.devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-10-28 13:52:13.219573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 13:52:13.251681: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 13:52:13.251839: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 13:52:13.252220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
with open('LSTM65.yaml', 'r') as file:
    model_config = safe_load(file)

In [5]:
model = tf.keras.models.load_model(
    './saved_models/{}'.format(model_config["Model_Name"]), 
    custom_objects = {
        'CosineDistance': CosineDistance,
        'ContrastiveLoss': ContrastiveLoss
    },
    compile=True,
    options=None
)
model.summary()

2022-10-28 13:52:13.761535: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 13:52:13.761712: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 13:52:13.761815: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 13:52:13.761950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-28 13:52:13.762053: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

2022-10-28 13:52:19.618714: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-28 13:52:19.627413: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-28 13:52:19.692277: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-28 13:52:19.700669: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-28 13:52:19.710110: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_sh

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 model (Functional)             (None, 65)           115065      ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

## Find equivalence groups

In [6]:
target_sequences = encode_pad_tag(preprocessInput(filename="../data/NP_subset_LB_common_names.csv", maxlen= model_config['MAXLEN'], noise= None, balance=False, reflexive = True), match = 1, distance = 0, maxlen = model_config['MAXLEN'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   latin_binomial  70 non-null     object
 1   common_name     70 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['latin_binomial', 'common_name'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 2 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtyp

In [7]:
equivalence_set = get_target_groups_from_connected_components(target_sequences)
equivalence_set

[{'ACAI', 'EUTERPE OLERACEA'},
 {'ALOE VERA'},
 {'APPLE CIDER VINEGAR', 'MALUS DOMESTICA', 'MALUS PUMILA'},
 {'ASHWAGANDA', 'WITHANIA SOMNIFERA'},
 {'BARLEY GRASS', 'HORDEUM VULGARE'},
 {'BEET ROOT', 'BETA VULGARIS'},
 {'ACTAEA RACEMOSA', 'BLACK COHOSH'},
 {'BLACK CUMIN', 'NIGELLA SATIVA'},
 {'BOSWELFIA SERRATA', 'BOSWELLIA'},
 {'BUTCHERSBROOM', 'RUSCUS ACULEATUS'},
 {'CATSCLAW', 'UNCARIA TOMENTOSA'},
 {'CHAMOMILE', 'MATRICARIA CHAMOMILLA'},
 {'BLACK CHERRY', 'PRUNUS SEROTINA'},
 {'CHLORELLA', 'CHLORELLA VULGARIS'},
 {'CINNAMOMUM CASSIA', 'CINNAMOMUM VERUM', 'CINNAMON'},
 {'CORDYCEPS', 'OPHIOCORDYCEPS SINENSIS'},
 {'CRANBERRY', 'VACCINIUM MACROCARPON'},
 {'ECHINACEA', 'ECHINACEA ANGUSTIFOLIA'},
 {'ECHINACEA PURPUREA', 'ECHINACEAPURPLE CONEFLOWER'},
 {'ELDERBERRY', 'SAMBUCUS NIGRA'},
 {'EVENING PRIMROSE OIL', 'OENOTHERA BIENNIS'},
 {'FENNEL', 'FOENICULUM VULGARE'},
 {'FENUGREEK', 'TRIGONELFA FOENUM'},
 {'FEVERFEW', 'TANACETUM PARTHENIUM'},
 {'FLAX SEED', 'LINUM USITATISSIMUM'},
 {'GARCI

# Preprocess targets

In [8]:
target_sequences = encode_pad_tag(preprocessInput(filename="../data/NP_subset_LB_common_names.csv", maxlen= model_config['MAXLEN'], noise= None, balance=False, reflexive = False), match = 1, distance = 0, maxlen = model_config['MAXLEN'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   latin_binomial  70 non-null     object
 1   common_name     70 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['latin_binomial', 'common_name'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 0 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 

In [9]:
target_sequences.rename(columns={"x": "Latin Binomial", "y": "Prefered Term"}).loc[:,["Latin Binomial", "Prefered Term"]].to_csv('./Novelty_Results/70_target_NPs_LB2PT.csv')

# Latin Binomial

In [10]:
LatinBinomials = target_sequences.iloc[:,[0,2]]
LatinBinomials.head(5)

,x,Processed_x
0,EUTERPE OLERACEA,"[5, 21, 20, 5, 18, 16, 5, 27, 15, 12, 5, 18, 1..."
1,ALOE VERA,"[1, 12, 15, 5, 27, 22, 5, 18, 1, 0, 0, 0, 0, 0..."
2,MALUS DOMESTICA,"[13, 1, 12, 21, 19, 27, 4, 15, 13, 5, 19, 20, ..."
3,MALUS PUMILA,"[13, 1, 12, 21, 19, 27, 16, 21, 13, 9, 12, 1, ..."
4,WITHANIA SOMNIFERA,"[23, 9, 20, 8, 1, 14, 9, 1, 27, 19, 15, 13, 14..."


# Common Names

In [11]:
CommonNames = target_sequences.iloc[:,[1,3]]
CommonNames.rename(columns={"y": "x", "Processed_y": "Processed_x"}, inplace = True)
CommonNames.head(5)

/tmp/ipykernel_259988/4021679550.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CommonNames.rename(columns={"y": "x", "Processed_y": "Processed_x"}, inplace = True)


,x,Processed_x
0,ACAI,"[1, 3, 1, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,ALOE VERA,"[1, 12, 15, 5, 27, 22, 5, 18, 1, 0, 0, 0, 0, 0..."
2,APPLE CIDER VINEGAR,"[1, 16, 16, 12, 5, 27, 3, 9, 4, 5, 18, 27, 22,..."
3,APPLE CIDER VINEGAR,"[1, 16, 16, 12, 5, 27, 3, 9, 4, 5, 18, 27, 22,..."
4,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ..."


#  Preprocess unmapped sequences

In [12]:
unmapped_sequences = preprocessInput("../data/unmapped-faers-drug-strings-Q12004-through-Q2-2021-extended-NP-vocab-09012022.tsv", maxlen= model_config['MAXLEN'], balance=False, reflexive = False, noise = None, sep="\t")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389386 entries, 0 to 389385
Data columns (total 1 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   drug_name_original  389385 non-null  object
dtypes: object(1)
memory usage: 3.0+ MB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['drug_name_original'], dtype='object')
	New names: Index(['x'], dtype='object')
Dropping row with empty cells:
	Dropped 1 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 7751 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 41849 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339785 entries, 0 to 339784
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  -------

In [13]:
# unmapped_sequences = unmapped_sequences.sample(n=30)
# unmapped_sequences.reset_index(inplace=True)
# unmapped_sequences = unmapped_sequences[["x"]]

In [14]:
unmapped_sequences.head(5)

,x
0,DULOXETINE MG EVERY NIGHT AT BEDTIME
1,ATELEC CLINIDIPINE
2,MAXIFLOXACIN
3,PRANDIN TAB
4,ST JOSHEPHS MG


In [15]:
unmapped_sequences = encode_pad_tag(unmapped_sequences, match = 1, distance = 0, maxlen = model_config['MAXLEN'])

Encoding and Padding: ----------------------------------
	Processing x
Tagging: -----------------------------------------------


## Master Record

In [16]:
UUID_MR = pd.DataFrame(columns=["Model_PT_UUID", "Model_LB_UUID", "Gestalt_PT_UUID" , "Gestalt_LB_UUID"])

# Model PT

In [17]:
results_MPT = find_Model_ranks(model, df = CommonNames, ranks = 20, targets = unmapped_sequences, equivalence_set = equivalence_set)

/home/israel/Documents/Academic/Research/Richard_Boyce/NP-FAERS-embed/experiment-4/ModelEvaluation.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rank{}".format(i)] = ""
/home/israel/Documents/Academic/Research/Richard_Boyce/NP-FAERS-embed/experiment-4/ModelEvaluation.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rank{}".format(i)] = ""
/home/israel/Documents/Academic/Research/Richard_Boyce/NP-FAERS-embed/experiment-4/ModelEvaluation.py:60: SettingWithCopyWarning: 
A value is trying to 

In [18]:
results_MPT = results_MPT.iloc[:, ~results_MPT.columns.isin(['Processed_x', 'Processed_y', 'Match', 'Distance'])]
results_MPT["x"] = results_MPT["x"].apply(lambda x: x + " Synonyms: " + str(next(list(y) for y in equivalence_set if x in y)))
uuids = []
for _ in range(results_MPT.index.size):
    uuids.append("".join(["[", str(uuid.uuid4()), "]"]))
UUID_MR["Model_PT_UUID"] = uuids
results_MPT["uuid"] = UUID_MR["Model_PT_UUID"] 

/tmp/ipykernel_259988/807129096.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_MPT["x"] = results_MPT["x"].apply(lambda x: x + " Synonyms: " + str(next(list(y) for y in equivalence_set if x in y)))


In [19]:
# results_MPT

In [20]:
for i in results_MPT.index:
    results_MPT.loc[i].to_csv("./Novelty_Results/Model/PT_{}.txt".format(i), sep='\n', header= False, index=False)

In [21]:
del results_MPT

----------------------

# MODEL LB

In [22]:
results_MLB = find_Model_ranks(model, df = LatinBinomials, ranks = 20, targets = unmapped_sequences, equivalence_set = equivalence_set)

/home/israel/Documents/Academic/Research/Richard_Boyce/NP-FAERS-embed/experiment-4/ModelEvaluation.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rank{}".format(i)] = ""
/home/israel/Documents/Academic/Research/Richard_Boyce/NP-FAERS-embed/experiment-4/ModelEvaluation.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rank{}".format(i)] = ""
/home/israel/Documents/Academic/Research/Richard_Boyce/NP-FAERS-embed/experiment-4/ModelEvaluation.py:60: SettingWithCopyWarning: 
A value is trying to 

In [23]:
results_MLB = results_MLB.iloc[:, ~results_MLB.columns.isin(['Processed_x', 'Processed_y', 'Match', 'Distance'])]
results_MLB["x"] = results_MLB["x"].apply(lambda x: x + " Synonyms: " + str(next(list(y) for y in equivalence_set if x in y)))
uuids = []
for _ in range(results_MLB.index.size):
    uuids.append("".join(["[", str(uuid.uuid4()), "]"]))
UUID_MR["Model_LB_UUID"] = uuids
results_MLB["uuid"] = UUID_MR["Model_LB_UUID"] 

/tmp/ipykernel_259988/1397636226.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_MLB["x"] = results_MLB["x"].apply(lambda x: x + " Synonyms: " + str(next(list(y) for y in equivalence_set if x in y)))


In [24]:
# results_MLB

In [25]:
for i in results_MLB.index:
    results_MLB.loc[i].to_csv("./Novelty_Results/Model/LB_{}.txt".format(i), sep='\n', header= False, index=False)

In [26]:
del results_MLB

------------------------------------------------

# GESTALT PT

In [27]:
results_GPT = find_Gesalt_ranks(df = CommonNames.loc[:, ['x', 'Processed_x']], ranks = 20, targets = unmapped_sequences, equivalence_set = equivalence_set, find_related_rank= False)

In [28]:
results_GPT = results_GPT.iloc[:, ~results_GPT.columns.isin(['Processed_x', 'Processed_y', 'Match', 'Distance'])]
results_GPT["x"] = results_GPT["x"].apply(lambda x: x + " Synonyms: " + str(next(list(y) for y in equivalence_set if x in y)))
uuids = []
for _ in range(results_GPT.index.size):
    uuids.append("".join(["[", str(uuid.uuid4()), "]"]))
UUID_MR["Gestalt_PT_UUID"] = uuids
results_GPT["uuid"] = UUID_MR["Gestalt_PT_UUID"] 

In [29]:
# results_GPT

In [30]:
for i in results_GPT.index:
    results_GPT.loc[i].to_csv("./Novelty_Results/Gestalt/PT_{}.txt".format(i), sep='\n', header= False, index=False)       

In [31]:
del results_GPT

------------------------------------------------

# GESTALT LB

In [32]:
results_GLB = find_Gesalt_ranks(df = LatinBinomials.loc[:, ['x', 'Processed_x']], ranks = 20, targets = unmapped_sequences, equivalence_set = equivalence_set, find_related_rank= False)

In [33]:
results_GLB = results_GLB.iloc[:, ~results_GLB.columns.isin(['Processed_x', 'Processed_y', 'Match', 'Distance'])]
results_GLB["x"] = results_GLB["x"].apply(lambda x: x + " Synonyms: " + str(next(list(y) for y in equivalence_set if x in y)))
uuids = []
for _ in range(results_GLB.index.size):
    uuids.append("".join(["[", str(uuid.uuid4()), "]"]))
UUID_MR["Gestalt_LB_UUID"] = uuids
results_GLB["uuid"] = UUID_MR["Gestalt_LB_UUID"] 

In [34]:
# results_GLB

In [35]:
for i in results_GLB.index:
    results_GLB.loc[i].to_csv("./Novelty_Results/Gestalt/LB_{}.txt".format(i), sep='\n', header= False, index=False)

In [36]:
del results_GLB

------------------------------------------------

In [37]:
UUID_MR.to_csv("./Novelty_Results/MasterRecords.csv")

In [38]:
# results["x"] = results["x"].apply(lambda x: x + " Synonyms: " + equivalence_set[equivalence_set['x'] == x]["y"].to_list() if equivalence_set[equivalence_set['x'] == x]["x"].any() else x)

In [39]:
# dUnique_df = pd.read_pickle("../data/dUnique_df.pkl")

In [40]:
# latin_2_common = preprocessInput('../data/vocabulary_LB_to_common_names_09082022.tsv', maxlen= model_config['MAXLEN'], reflexive=True, balance = False, sep='\t')

In [41]:
# common_2_latin = preprocessInput('../data/positive-unmapped-pairs-common-name-or-latin-binomial-copies.tsv', maxlen= model_config['MAXLEN'], reflexive=True, balance = False, sep='\t')

In [42]:
# gsrs_2_common = preprocessInput('../data/positive-unmapped-pairs-gsrs-name-to-common-name.tsv', maxlen= model_config['MAXLEN'], reflexive=True, balance = False, sep='\t')

In [43]:
# equivalence_set = pd.concat([latin_2_common, common_2_latin, gsrs_2_common], ignore_index=True)
# equivalence_set.drop_duplicates(subset=['x', 'y'], inplace=True)